In [1]:
#!pip install --upgrade pika

In [1]:
import pika
import sys
pika.__version__

'1.1.0'

In [3]:
!sudo rabbitmqctl add_user myguest myguestpwd
!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

Creating user "myguest" ...
Error: user_already_exists: myguest
Setting permissions for user "myguest" in vhost "/" ...


In [2]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


In [3]:
!sudo rabbitmqctl list_bindings

Listing bindings ...
	exchange	amq.gen-L1thtIGEsmOb6jTGRBD04g	queue	amq.gen-L1thtIGEsmOb6jTGRBD04g	[]
	exchange	amq.gen-W1f-cSg2hOe4Lh95CqRxxw	queue	amq.gen-W1f-cSg2hOe4Lh95CqRxxw	[]
	exchange	amq.gen-lrOH1y0b4n955khMfG8Wsg	queue	amq.gen-lrOH1y0b4n955khMfG8Wsg	[]
	exchange	amq.gen-tzKD3AstQV43tTeDRaW3aw	queue	amq.gen-tzKD3AstQV43tTeDRaW3aw	[]
ping	exchange	amq.gen-W1f-cSg2hOe4Lh95CqRxxw	queue	amq.gen-W1f-cSg2hOe4Lh95CqRxxw	[]


In [4]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
URL='localhost'
#URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)

In [ ]:
class FibonacciRpcClient(object):

    def __init__(self):
        #self.connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))

        self.connection = pika.BlockingConnection(parameters)
        self.channel = self.connection.channel() #This channel will be used to send and receive ping messages

        myqueue = self.channel.queue_declare('', exclusive=True)
        print("myqueue:", myqueue)
        
        self.callback_queue = myqueue.method.queue #The queue we just created

        print("myqueue.method:", myqueue.method)
        
        # Let us start waiting for messages (The results of our call). 
        #When we recevie a message, we will the on_response call back function
        self.channel.basic_consume(
            queue=self.callback_queue, #the queue we created above
            on_message_callback=self.on_response, #The callback function to run when we receive a message from the workers
            auto_ack=True)

    def on_response(self, ch, method, props, body):
        if self.corr_id == props.correlation_id: #Check if our correlation id is the same as we get in the message
            self.response = body

    def call(self, n):
        self.response = None
        self.corr_id = str(uuid.uuid4()) # This is where we create our unique correlation id
        print('corr_id:', self.corr_id) #We will use the correlaion_id when sending and receiving a message
        
        # Now we are ready to send our message to the workers
        self.channel.basic_publish(
            exchange='', #send the message on default exchange
            routing_key='rpc_queue', #send the meassage on "rpc_queue"
            properties=pika.BasicProperties(
                reply_to=self.callback_queue, #message will be received on the queue we created in init
                correlation_id=self.corr_id,
            ),
            body=str(n))
        #Waiting for the results
        while self.response is None:
            self.connection.process_data_events()
        return int(self.response)

In [5]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
    
URL='localhost'
#URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)

exchangeName = 'ping'

In [6]:
def on_response(self, ch, method, props, body):
    print(body)

In [7]:
myconnection = pika.BlockingConnection(parameters)

In [8]:
#Create a channel for sending out ping message
mychannel = myconnection.channel()

mychannel.exchange_declare(exchange=exchangeName, exchange_type='fanout')

#ping all workers
#for fanout type exchange, routing key/queue is useless
properties = pika.spec.BasicProperties(content_type = "text/plain", delivery_mode = 1)
message = "Ping EveryOne!"

#channel.basic_publish(exchange = exchangeName, routing_key = "", body = message, properties = properties)


queue_ping = mychannel.queue_declare('')
print("myqueue:", queue_ping)
callback_queue = queue_ping.method.queue #The queue we just created
print("callback_queue", callback_queue)

myqueue: <METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=0', 'queue=amq.gen-LSrhVaeNvLDFlD1mjkQVUg'])>"])>
callback_queue amq.gen-LSrhVaeNvLDFlD1mjkQVUg


In [9]:
mychannel.basic_consume(
            queue=callback_queue, #the queue we created above
            on_message_callback=on_response, #The callback function to run when we receive a message from the workers
            auto_ack=True)

'ctag1.07bd5f38545c47ab9d2a9bdc88337cb1'

In [10]:
#disconnect
#connection.close()

In [11]:
mychannel.basic_publish(
            exchange=exchangeName,
            routing_key="",
            properties=pika.BasicProperties(
                reply_to=callback_queue,
            ),
            body=message)

**After this, we should see that all workers received the same message**

In [7]:
!sudo rabbitmqctl list_bindings

Listing bindings ...
	exchange	amq.gen-gf_WYyWCqa0YVkH8hbihPA	queue	amq.gen-gf_WYyWCqa0YVkH8hbihPA	[]
	exchange	hello2	queue	hello2	[]
	exchange	rpc_queue	queue	rpc_queue	[]
	exchange	task_queue	queue	task_queue	[]
direct_logs	exchange	task_queue	queue	NYC	[]
direct_logs	exchange	task_queue	queue	Pittsburgh	[]
direct_logs	exchange	task_queue	queue	Washington	[]
ping	exchange	amq.gen-gf_WYyWCqa0YVkH8hbihPA	queue	amq.gen-gf_WYyWCqa0YVkH8hbihPA	[]


In [8]:
!sudo rabbitmqctl list_exchanges

Listing exchanges ...
	direct
amq.direct	direct
amq.fanout	fanout
amq.headers	headers
amq.match	headers
amq.rabbitmq.log	topic
amq.rabbitmq.trace	topic
amq.topic	topic
direct_logs	direct
logs	fanout
ping	fanout
